#Librerías y paquetes

In [1]:
!pip install scikit-surprise

In [22]:
import numpy as np
import pandas as pd
import sqlite3

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
####Paquete para sistemas de recomendación surprise
###Puede generar problemas en instalación local de pyhton. Genera error instalando con pip
#### probar que les funcione para la próxima clase

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [3]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_moviesF')
cur=conn.cursor()

#Verificar tablas creadas en el script
# Conectarse a la base de datos
conn = sqlite3.connect('db_moviesF')

# Crear un cursor
cursor = conn.cursor()

# Obtener una lista de tablas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])

In [4]:
# Verificar las columnas de la tabla ratings_final
cursor.execute("PRAGMA table_info(ratings_final)")

columns_ratings_final = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_ratings_final:
    print(column[1], "-", column[2])


user_id - INT
movie_id - INT
rating - REAL


In [5]:
# Verificar las columnas de la tabla ratings
cursor.execute("PRAGMA table_info(ratings)")

columns_ratings = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_ratings:
    print(column[1], "-", column[2])


userId - INTEGER
movieId - INTEGER
rating - REAL
timestamp - INTEGER


In [6]:
# Verificar las columnas de la tabla movies
cursor.execute("PRAGMA table_info(movies)")

columns_movies = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_movies:
    print(column[1], "-", column[2])


movieId - INTEGER
title - TEXT
genres - TEXT
year - INTEGER


#Sistema de recomendación basado en contenido KNN

##Con base en todo lo visto por el usuario

In [7]:
##### cargar dataframe escalado y con dummies ###
movies_dum2= joblib.load('salidas_bmovies_dum2.joblib')

In [8]:
#### seleccionar usuario para recomendaciones ####
usuarios=pd.read_sql('select distinct (user_id) as user_id from ratings_final',conn)
print (usuarios)
user_id=1 ### para ejemplo manual

     user_id
0          1
1          4
2          6
3          7
4         10
..       ...
361      603
362      604
363      605
364      607
365      608

[366 rows x 1 columns]


In [9]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id,})

    ###convertir ratings del usuario a array
    l_movies_w=ratings['movie_id'].to_numpy()

In [10]:
# Cargar los datos de la tabla 'ratings' en un DataFrame
ratings = pd.read_sql('SELECT * FROM ratings', conn)

cursor.execute("DROP TABLE IF EXISTS l_movies_w")

# crear la tabla l_movies_w en la base de datos
consulta_sql = """
CREATE TABLE l_movies_w AS
SELECT movieId AS movie_id FROM ratings;
"""

cursor.execute(consulta_sql)

# Leer los datos de la tabla l_movies_r en un DataFrame de Pandas
l_movies_w = pd.read_sql('SELECT * FROM l_movies_w', conn)


In [11]:
# Cargar los datos de la tabla 'movies' en un DataFrame
movies = pd.read_sql('SELECT * FROM movies', conn)

# Agregar las columnas 'movieId' y 'title' al DataFrame 'movies_dum2'
movies_dum2['movie_id'] = movies['movieId']
movies_dum2['title'] = movies['title']

print(movies_dum2)

       year_sc  genres_(no genres listed)  genres_Action  \
0     0.988603                          0              0   
1     0.988603                          0              0   
2     0.988603                          0              0   
3     0.988603                          0              0   
4     0.988603                          0              0   
...        ...                        ...            ...   
9737  0.999504                          0              0   
9738  0.999504                          0              0   
9739  0.999504                          0              0   
9740  1.000000                          0              0   
9741  0.986620                          0              0   

      genres_Action|Adventure  genres_Action|Adventure|Animation  \
0                           0                                  0   
1                           0                                  0   
2                           0                                  0   
3      

In [12]:
# Verificar si hay valores duplicados en la columna 'movie_id' de la tabla l_movies_w
duplicates_count = l_movies_w['movie_id'].duplicated().sum()

if duplicates_count == 0:
    print("No hay valores duplicados en la columna 'movie_id' de l_movies_w.")
else:
    print(f"Se encontraron {duplicates_count} valores duplicados en la columna 'movie_id' de l_movies_w.")


Se encontraron 91112 valores duplicados en la columna 'movie_id' de l_movies_w.


In [42]:
# Como hay demasiados valores duplicados, se procede a eliminar valores duplicados en la columna 'movie_id' de la tabla l_movies_w
l_movies_w = l_movies_w.drop_duplicates(subset=['movie_id'])

# Verificar si hay valores duplicados después de la eliminación
duplicates_count_after_removal = l_movies_w['movie_id'].duplicated().sum()

if duplicates_count_after_removal == 0:
    print("Se eliminaron los valores duplicados con éxito.")
else:
    print("Aún quedan valores duplicados después de eliminarlos.")

Se eliminaron los valores duplicados con éxito.


In [43]:
#Verificamos el tamaño del dataframe resultante y aún quedan 9724 registros
print(l_movies_w.shape)

(9724, 1)


In [44]:
# Filtrar películas calificadas por el usuario en movies_dum2
movies_w = movies_dum2[movies_dum2['movie_id'].isin(l_movies_w)]

In [45]:
#Verificamos las dimensiones de movies_w
print (movies_w.shape)

#########VERIFICAR POR QUÉ NO HAY NINGUNA COINCIDENCIA, el dataframe no puede quedar vacío#############

(0, 954)


In [17]:
## eliminar columna nombre y movie_id
movies_w=movies_w.drop(columns=['movie_id','title'])
movies_w["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
##centroide o perfil del usuario
centroide=movies_w.groupby("indice").mean()

In [26]:
#Verificamos el tamaño de centroide
print(centroide)

######### ESTÁ ASOCIADO AL PROBLEMA CON movies_w, el dataframe no puede quedar vacío porque se utiliza en el modelo#############

Empty DataFrame
Columns: [year_sc, genres_(no genres listed), genres_Action, genres_Action|Adventure, genres_Action|Adventure|Animation, genres_Action|Adventure|Animation|Children, genres_Action|Adventure|Animation|Children|Comedy, genres_Action|Adventure|Animation|Children|Comedy|Fantasy, genres_Action|Adventure|Animation|Children|Comedy|IMAX, genres_Action|Adventure|Animation|Children|Comedy|Romance, genres_Action|Adventure|Animation|Children|Comedy|Sci-Fi, genres_Action|Adventure|Animation|Children|Comedy|Sci-Fi|IMAX, genres_Action|Adventure|Animation|Children|Comedy|Western, genres_Action|Adventure|Animation|Children|Fantasy, genres_Action|Adventure|Animation|Children|Fantasy|Sci-Fi, genres_Action|Adventure|Animation|Children|Sci-Fi, genres_Action|Adventure|Animation|Comedy, genres_Action|Adventure|Animation|Comedy|Crime|Mystery, genres_Action|Adventure|Animation|Comedy|Fantasy|Mystery|Sci-Fi, genres_Action|Adventure|Animation|Comedy|Fantasy|Sci-Fi, genres_Action|Adventure|Animatio

In [46]:
 ### filtrar películas no vistas
movies_nw=movies_dum2[~movies_dum2['movie_id'].isin(l_movies_w)]
## eliminar nombre e id
movies_nw=movies_nw.drop(columns=['movie_id','title'])

In [47]:
#Verificamos el tamaño de movies_nw
print (movies_nw.shape)

(9742, 952)


In [48]:
### Entrenar modelo
model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(movies_nw)
dist, idlist = model.kneighbors(centroide)

###############DEBE SOLUCIONARSE EL DATAFRAME RESULTANTE DE CENTROIDE####################

ValueError: Found array with 0 sample(s) (shape=(0, 952)) while a minimum of 1 is required by NearestNeighbors.

In [49]:
ids=idlist[0] ### queda en un array anidado, para sacarlo
recomend_b=movies.loc[ids][['title','movieId']]
vistas=movies[movies['movieId'].isin(l_movies_w)][['title','movieId']]

NameError: name 'idlist' is not defined

In [ ]:
return recomend_b

In [ ]:
recomendar(52853)


print(interact(recomendar))

##Sistema de recomendación filtro colaborativo

In [28]:
### datos originales en pandas
## knn solo sirve para calificaciones explicitas
ratings=pd.read_sql('select * from ratings_final where rating>0', conn)

In [29]:
####los datos deben ser leídos en un formato especial para surprise
reader = Reader(rating_scale=(0, 10)) ### la escala de la calificación
###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader)

In [30]:
#####Existen varios modelos
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}
###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación


In [31]:
#### for para probar varios modelos ##########
model=models[1]
for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

In [32]:
performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.635812,0.837393,0.156125,1.347948
knns.KNNWithZScore,0.641824,0.846954,0.113160,1.301293
knns.KNNWithMeans,0.645216,0.847668,0.075396,1.158665
knns.KNNBasic,0.677415,0.893412,0.052806,0.937332


In [33]:
# Se escoge el mejor knn withmeans
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}
             }
## min support es la cantidad de items o usuarios que necesita para calcular recomendación
## name medidas de distancia

### se afina si es basado en usuario o basado en ítem

In [34]:
gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)

In [35]:
gridsearchKNNWithMeans.fit(data)

In [36]:
gridsearchKNNWithMeans.best_params["rmse"]
gridsearchKNNWithMeans.best_score["rmse"]
gs_model=gridsearchKNNWithMeans.best_estimator['rmse'] ### mejor estimador de gridsearch

## Entrenar con todos los datos y Realizar predicciones con el modelo afinado

In [37]:
trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrenamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [38]:
predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y las películas que no han visto
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)

127069

In [39]:
predictions = gs_model.test(predset) ### función muy pesada, hace las predicciones de rating para todas las películas que no ha visto un usuario
### la funcion test recibe un test set construido con build_test method, o el que genera crosvalidate

In [40]:
####### predicción para una película puntual
model.predict(uid=269397, iid='0446353205',r_ui='') ### uid debía estar en número e isb en comillas

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

,uid,iid,r_ui,est,details
111996,543,318,3.716676,5.234745,"{'actual_k': 38, 'was_impossible': False}"
91703,429,318,3.716676,5.232612,"{'actual_k': 25, 'was_impossible': False}"
124516,597,318,3.716676,5.209582,"{'actual_k': 40, 'was_impossible': False}"
108921,523,858,3.716676,5.204872,"{'actual_k': 37, 'was_impossible': False}"
108954,523,1276,3.716676,5.187059,"{'actual_k': 34, 'was_impossible': False}"
...,...,...,...,...,...
60526,287,2701,3.716676,0.725391,"{'actual_k': 40, 'was_impossible': False}"
34425,153,173,3.716676,0.671468,"{'actual_k': 40, 'was_impossible': False}"
34183,153,2054,3.716676,0.652668,"{'actual_k': 40, 'was_impossible': False}"
34294,153,435,3.716676,0.462566,"{'actual_k': 40, 'was_impossible': False}"


##Función para recomendar las 10 películas con mejores predicciones y llevar a la base de datos para consultar el resto de información

In [41]:
def recomendaciones(user_id,n_recomend=10):

    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendadas = predictions_userID[['iid','est']]
    recomendadas.to_sql('reco',conn,if_exists="replace")

    recomendadas=pd.read_sql('''select a.*, b.movie_title
                             from reco a left join movies_final b
                             on a.iid=b.movie_id ''', conn)

    return(recomendadas)



recomendaciones(user_id=55490,n_recomend=10)

,index,iid,est,movie_title
